In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
from datetime import date, datetime
from typing import List, Optional
from docx import Document
from docx.shared import Inches
import json

# Helper functions for JSON serialization/deserialization
def date_to_str(obj):
    if isinstance(obj, date):
        return obj.isoformat()
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

def str_to_date(dct):
    for key, value in dct.items():
        if key in ["dob", "assessment_date", "reassessment_date"] and value:
            dct[key] = date.fromisoformat(value)
    return dct

class MedicalHistory:
    def __init__(self, conditions: List[str], surgeries: List[str], allergies: List[str]):
        self.conditions = conditions
        self.surgeries = surgeries
        self.allergies = allergies

    def __str__(self):
        return f"Conditions: {', '.join(self.conditions)}, Surgeries: {', '.join(self.surgeries)}, Allergies: {', '.join(self.allergies)}"

    def to_dict(self):
        return {
            "conditions": self.conditions,
            "surgeries": self.surgeries,
            "allergies": self.allergies
        }

class Diagnosis:
    def __init__(self, diagnosis: str, icd_code: str):
        self.diagnosis = diagnosis
        self.icd_code = icd_code

    def __str__(self):
        return f"Diagnosis: {self.diagnosis} (ICD: {self.icd_code})"

    def to_dict(self):
        return {
            "diagnosis": self.diagnosis,
            "icd_code": self.icd_code
        }

class Assessment:
    def __init__(self, findings: str, assessment_date: date, subjective_symptoms: str, objective_findings: str, history: str):
        self.findings = findings
        self.assessment_date = assessment_date
        self.subjective_symptoms = subjective_symptoms
        self.objective_findings = objective_findings
        self.history = history

    def __str__(self):
        return (f"Assessment on {self.assessment_date}: {self.findings}\n"
                f"Subjective: {self.subjective_symptoms}\n"
                f"Objective: {self.objective_findings}\n"
                f"History: {self.history}")

    def to_dict(self):
        return {
            "findings": self.findings,
            "assessment_date": self.assessment_date,
            "subjective_symptoms": self.subjective_symptoms,
            "objective_findings": self.objective_findings,
            "history": self.history
        }

class Reassessment:
    def __init__(self, date: date, progress_notes: str, objective_findings: str, recommendations: str):
        self.reassessment_date = date
        self.progress_notes = progress_notes
        self.objective_findings = objective_findings
        self.recommendations = recommendations

    def __str__(self):
        return (f"Reassessment on {self.reassessment_date}: {self.progress_notes}\n"
                f"Objective: {self.objective_findings}\n"
                f"Recommendations: {self.recommendations}")

    def to_dict(self):
        return {
            "reassessment_date": self.reassessment_date,
            "progress_notes": self.progress_notes,
            "objective_findings": self.objective_findings,
            "recommendations": self.recommendations
        }

class TreatmentPlan:
    def __init__(self, exercises: List[str], therapies: List[str], modalities: List[str], interventions: str, home_exercise_program: str):
        self.exercises = exercises
        self.therapies = therapies
        self.modalities = modalities
        self.interventions = interventions
        self.home_exercise_program = home_exercise_program

    def __str__(self):
        return (f"Exercises: {', '.join(self.exercises)}\n"
                f"Therapies: {', '.join(self.therapies)}\n"
                f"Modalities: {', '.join(self.modalities)}\n"
                f"Interventions: {self.interventions}\n"
                f"Home Program: {self.home_exercise_program}")

    def to_dict(self):
        return {
            "exercises": self.exercises,
            "therapies": self.therapies,
            "modalities": self.modalities,
            "interventions": self.interventions,
            "home_exercise_program": self.home_exercise_program
        }

class Patient:
    def __init__(self, name: str, contact: str, dob: date, gender: str, medical_aid: str, medical_aid_number: str,
                 occupation: str, referral_diagnosis: str, physiotherapist_name: str, medical_history: MedicalHistory,
                 diagnosis: Diagnosis, treatment_plan: TreatmentPlan):
        self.name = name
        self.contact = contact
        self.dob = dob
        self.gender = gender
        self.medical_aid = medical_aid
        self.medical_aid_number = medical_aid_number
        self.occupation = occupation
        self.referral_diagnosis = referral_diagnosis
        self.physiotherapist_name = physiotherapist_name
        self.medical_history = medical_history
        self.diagnosis = diagnosis
        self.treatment_plan = treatment_plan
        self.assessments: List[Assessment] = []
        self.reassessments: List[Reassessment] = []

    def add_assessment(self, assessment: Assessment):
        self.assessments.append(assessment)

    def add_reassessment(self, reassessment: Reassessment):
        self.reassessments.append(reassessment)

    def __str__(self):
        return (f"Patient: {self.name}, DOB: {self.dob}, Gender: {self.gender}, Contact: {self.contact}\n"
                f"Medical Aid: {self.medical_aid}, Number: {self.medical_aid_number}\n"
                f"Occupation: {self.occupation}, Referral Diagnosis: {self.referral_diagnosis}\n"
                f"Physiotherapist: {self.physiotherapist_name}\n"
                f"Medical History: {self.medical_history}\n"
                f"Diagnosis: {self.diagnosis}\n"
                f"Treatment Plan: {self.treatment_plan}\n"
                f"Assessments: {len(self.assessments)}, Reassessments: {len(self.reassessments)}")

    def to_dict(self):
        return {
            "name": self.name,
            "contact": self.contact,
            "dob": self.dob,
            "gender": self.gender,
            "medical_aid": self.medical_aid,
            "medical_aid_number": self.medical_aid_number,
            "occupation": self.occupation,
            "referral_diagnosis": self.referral_diagnosis,
            "physiotherapist_name": self.physiotherapist_name,
            "medical_history": self.medical_history.to_dict(),
            "diagnosis": self.diagnosis.to_dict(),
            "treatment_plan": self.treatment_plan.to_dict(),
            "assessments": [assessment.to_dict() for assessment in self.assessments],
            "reassessments": [reassessment.to_dict() for reassessment in self.reassessments]
        }

    @classmethod
    def from_dict(cls, data):
        medical_history = MedicalHistory(**data["medical_history"])
        diagnosis = Diagnosis(**data["diagnosis"])
        treatment_plan = TreatmentPlan(**data["treatment_plan"])
        patient = cls(
            data["name"], data["contact"], data["dob"], data["gender"], data["medical_aid"],
            data["medical_aid_number"], data["occupation"], data["referral_diagnosis"],
            data["physiotherapist_name"], medical_history, diagnosis, treatment_plan
        )
        patient.assessments = [Assessment(**assessment) for assessment in data["assessments"]]
        patient.reassessments = [Reassessment(**reassessment) for reassessment in data["reassessments"]]
        return patient

class PhysioApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Physiotherapy Report Generator")
        self.patients = {}
        self.current_patient = None
        self.patient_counter = 1
        self.patient_var = tk.StringVar()
        self.patient_var.set("Select Patient")

        # Create menu bar
        self.menu_bar = tk.Menu(self.root)
        self.root.config(menu=self.menu_bar)

        file_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="File", menu=file_menu)
        file_menu.add_command(label="Save Patients", command=self.save_patients)
        file_menu.add_command(label="Load Patients", command=self.load_patients)
        file_menu.add_separator()
        file_menu.add_command(label="Exit", command=self.root.quit)

        patient_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="Patient Management", menu=patient_menu)
        patient_menu.add_command(label="Add Patient", command=lambda: self.show_frame("patient"))
        patient_menu.add_command(label="View Patient Details", command=self.show_details)

        assess_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="Assessments & Progress", menu=assess_menu)
        assess_menu.add_command(label="Add Assessment", command=lambda: self.show_frame("assessment"))
        assess_menu.add_command(label="Add Reassessment", command=lambda: self.show_frame("assessment"))

        report_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="Reporting", menu=report_menu)
        report_menu.add_command(label="Generate Patient Report", command=self.generate_word_report)

        # Create main container for frames
        self.container = tk.Frame(self.root)
        self.container.grid(row=0, column=0, sticky="nsew")

        self.frames = {}
        self.create_patient_frame()
        self.create_assessment_frame()
        self.show_frame("patient")

        self.load_patients()

    def create_patient_frame(self):
        frame = tk.Frame(self.container)
        self.frames["patient"] = frame

        tk.Label(frame, text="Select Patient:").grid(row=0, column=0, padx=5, pady=5)
        self.patient_menu = tk.OptionMenu(frame, self.patient_var, *self.get_patient_names(), command=self.select_patient)
        self.patient_menu.grid(row=0, column=1, padx=5, pady=5, sticky="ew")

        tk.Label(frame, text="Patient Name:").grid(row=1, column=0, padx=5, pady=5)
        self.name_entry = tk.Entry(frame)
        self.name_entry.grid(row=1, column=1, padx=5, pady=5)

        tk.Label(frame, text="Contact (Email):").grid(row=2, column=0, padx=5, pady=5)
        self.contact_entry = tk.Entry(frame)
        self.contact_entry.grid(row=2, column=1, padx=5, pady=5)

        tk.Label(frame, text="DOB (YYYY-MM-DD):").grid(row=3, column=0, padx=5, pady=5)
        self.dob_entry = tk.Entry(frame)
        self.dob_entry.grid(row=3, column=1, padx=5, pady=5)

        tk.Label(frame, text="Gender:").grid(row=4, column=0, padx=5, pady=5)
        self.gender_entry = tk.Entry(frame)
        self.gender_entry.grid(row=4, column=1, padx=5, pady=5)

        tk.Label(frame, text="Medical Aid:").grid(row=5, column=0, padx=5, pady=5)
        self.medical_aid_entry = tk.Entry(frame)
        self.medical_aid_entry.grid(row=5, column=1, padx=5, pady=5)

        tk.Label(frame, text="Medical Aid Number:").grid(row=6, column=0, padx=5, pady=5)
        self.medical_aid_number_entry = tk.Entry(frame)
        self.medical_aid_number_entry.grid(row=6, column=1, padx=5, pady=5)

        tk.Label(frame, text="Occupation:").grid(row=7, column=0, padx=5, pady=5)
        self.occupation_entry = tk.Entry(frame)
        self.occupation_entry.grid(row=7, column=1, padx=5, pady=5)

        tk.Label(frame, text="Referral Diagnosis:").grid(row=8, column=0, padx=5, pady=5)
        self.referral_diagnosis_entry = tk.Entry(frame)
        self.referral_diagnosis_entry.grid(row=8, column=1, padx=5, pady=5)

        tk.Label(frame, text="Physiotherapist Name:").grid(row=9, column=0, padx=5, pady=5)
        self.physiotherapist_name_entry = tk.Entry(frame)
        self.physiotherapist_name_entry.grid(row=9, column=1, padx=5, pady=5)

        tk.Button(frame, text="Add Patient", command=self.add_patient).grid(row=10, column=0, columnspan=2, pady=10)
        self.details_text = tk.Text(frame, height=10, width=50)
        self.details_text.grid(row=11, column=0, columnspan=2, padx=5, pady=5)

    def create_assessment_frame(self):
        frame = tk.Frame(self.container)
        self.frames["assessment"] = frame

        tk.Label(frame, text="Select Patient:").grid(row=0, column=0, padx=5, pady=5)
        self.assess_patient_menu = tk.OptionMenu(frame, self.patient_var, *self.get_patient_names(), command=self.select_patient)
        self.assess_patient_menu.grid(row=0, column=1, padx=5, pady=5, sticky="ew")

        tk.Label(frame, text="Assessment Findings:").grid(row=1, column=0, padx=5, pady=5)
        self.assessment_findings_entry = tk.Text(frame, height=3, width=40)
        self.assessment_findings_entry.grid(row=1, column=1, padx=5, pady=5)

        tk.Label(frame, text="Subjective Symptoms:").grid(row=2, column=0, padx=5, pady=5)
        self.subjective_symptoms_entry = tk.Text(frame, height=3, width=40)
        self.subjective_symptoms_entry.grid(row=2, column=1, padx=5, pady=5)

        tk.Label(frame, text="Objective Findings:").grid(row=3, column=0, padx=5, pady=5)
        self.objective_findings_entry = tk.Text(frame, height=3, width=40)
        self.objective_findings_entry.grid(row=3, column=1, padx=5, pady=5)

        tk.Label(frame, text="History:").grid(row=4, column=0, padx=5, pady=5)
        self.history_entry = tk.Text(frame, height=3, width=40)
        self.history_entry.grid(row=4, column=1, padx=5, pady=5)

        tk.Button(frame, text="Add Assessment", command=self.add_assessment).grid(row=5, column=0, columnspan=2, pady=10)

        tk.Label(frame, text="Reassessment Progress Notes:").grid(row=6, column=0, padx=5, pady=5)
        self.reassessment_progress_entry = tk.Text(frame, height=3, width=40)
        self.reassessment_progress_entry.grid(row=6, column=1, padx=5, pady=5)

        tk.Label(frame, text="Reassessment Objective Findings:").grid(row=7, column=0, padx=5, pady=5)
        self.reassessment_objective_entry = tk.Text(frame, height=3, width=40)
        self.reassessment_objective_entry.grid(row=7, column=1, padx=5, pady=5)

        tk.Label(frame, text="Recommendations:").grid(row=8, column=0, padx=5, pady=5)
        self.recommendations_entry = tk.Text(frame, height=3, width=40)
        self.recommendations_entry.grid(row=8, column=1, padx=5, pady=5)

        tk.Button(frame, text="Add Reassessment", command=self.add_reassessment).grid(row=9, column=0, columnspan=2, pady=10)

    def show_frame(self, frame_name):
        frame = self.frames.get(frame_name)
        if frame:
            frame.grid(row=0, column=0, sticky="nsew")
            for other_frame in self.frames.values():
                if other_frame != frame:
                    other_frame.grid_remove()

    def get_patient_names(self):
        return [f"{pid}: {patient.name}" for pid, patient in self.patients.items()] or ["No patients available"]

    def select_patient(self, *args):
        selected = self.patient_var.get()
        if selected == "No patients available" or selected == "Select Patient":
            self.current_patient = None
            self.details_text.delete(1.0, tk.END)
            self.name_entry.delete(0, tk.END)
            self.contact_entry.delete(0, tk.END)
            self.dob_entry.delete(0, tk.END)
            self.gender_entry.delete(0, tk.END)
            self.medical_aid_entry.delete(0, tk.END)
            self.medical_aid_number_entry.delete(0, tk.END)
            self.occupation_entry.delete(0, tk.END)
            self.referral_diagnosis_entry.delete(0, tk.END)
            self.physiotherapist_name_entry.delete(0, tk.END)
            return
        
        pid = selected.split(":")[0].strip()
        if pid not in self.patients:
            messagebox.showerror("Error", "Invalid patient selection!")
            self.current_patient = None
            return
        
        self.current_patient = self.patients[pid]
        self.name_entry.delete(0, tk.END)
        self.name_entry.insert(0, self.current_patient.name)
        self.contact_entry.delete(0, tk.END)
        self.contact_entry.insert(0, self.current_patient.contact)
        self.dob_entry.delete(0, tk.END)
        self.dob_entry.insert(0, self.current_patient.dob.isoformat())
        self.gender_entry.delete(0, tk.END)
        self.gender_entry.insert(0, self.current_patient.gender)
        self.medical_aid_entry.delete(0, tk.END)
        self.medical_aid_entry.insert(0, self.current_patient.medical_aid)
        self.medical_aid_number_entry.delete(0, tk.END)
        self.medical_aid_number_entry.insert(0, self.current_patient.medical_aid_number)
        self.occupation_entry.delete(0, tk.END)
        self.occupation_entry.insert(0, self.current_patient.occupation)
        self.referral_diagnosis_entry.delete(0, tk.END)
        self.referral_diagnosis_entry.insert(0, self.current_patient.referral_diagnosis)
        self.physiotherapist_name_entry.delete(0, tk.END)
        self.physiotherapist_name_entry.insert(0, self.current_patient.physiotherapist_name)
        self.show_details()

    def update_patient_menu(self):
        for menu in [self.patient_menu, self.assess_patient_menu]:
            menu["menu"].delete(0, "end")
            for name in self.get_patient_names():
                menu["menu"].add_command(label=name, command=lambda n=name: self.patient_var.set(n))
        if self.current_patient:
            for pid, patient in self.patients.items():
                if patient == self.current_patient:
                    self.patient_var.set(f"{pid}: {patient.name}")
                    break
        else:
            self.patient_var.set("Select Patient")

    def add_patient(self):
        try:
            name = self.name_entry.get()
            contact = self.contact_entry.get()
            dob_str = self.dob_entry.get()
            gender = self.gender_entry.get()
            medical_aid = self.medical_aid_entry.get()
            medical_aid_number = self.medical_aid_number_entry.get()
            occupation = self.occupation_entry.get()
            referral_diagnosis = self.referral_diagnosis_entry.get()
            physiotherapist_name = self.physiotherapist_name_entry.get()
            
            if not all([name, contact, dob_str, gender, referral_diagnosis, physiotherapist_name]):
                messagebox.showerror("Error", "Required fields: Name, Contact, DOB, Gender, Referral Diagnosis, Physiotherapist Name!")
                return
            
            try:
                dob = date.fromisoformat(dob_str)
            except ValueError:
                messagebox.showerror("Error", "Invalid DOB format! Use YYYY-MM-DD")
                return

            medical_history = MedicalHistory(["none"], ["none"], ["none"])
            diagnosis = Diagnosis(referral_diagnosis, "N/A")
            treatment_plan = TreatmentPlan(["none"], ["none"], ["none"], "Initial treatment", "Home exercises prescribed")

            self.current_patient = Patient(
                name, contact, dob, gender, medical_aid or "N/A", medical_aid_number or "N/A",
                occupation or "N/A", referral_diagnosis, physiotherapist_name, medical_history,
                diagnosis, treatment_plan
            )
            patient_id = f"patient_{self.patient_counter:03d}"
            self.patients[patient_id] = self.current_patient
            self.patient_counter += 1

            self.update_patient_menu()
            messagebox.showinfo("Success", f"Patient {name} added with ID {patient_id}!")
            
            self.name_entry.delete(0, tk.END)
            self.contact_entry.delete(0, tk.END)
            self.dob_entry.delete(0, tk.END)
            self.gender_entry.delete(0, tk.END)
            self.medical_aid_entry.delete(0, tk.END)
            self.medical_aid_number_entry.delete(0, tk.END)
            self.occupation_entry.delete(0, tk.END)
            self.referral_diagnosis_entry.delete(0, tk.END)
            self.physiotherapist_name_entry.delete(0, tk.END)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to add patient: {str(e)}")

    def add_assessment(self):
        if not self.current_patient:
            messagebox.showerror("Error", "No patient selected!")
            return
        
        findings = self.assessment_findings_entry.get("1.0", tk.END).strip()
        subjective_symptoms = self.subjective_symptoms_entry.get("1.0", tk.END).strip()
        objective_findings = self.objective_findings_entry.get("1.0", tk.END).strip()
        history = self.history_entry.get("1.0", tk.END).strip()
        
        if not all([findings, subjective_symptoms, objective_findings]):
            messagebox.showerror("Error", "Findings, Subjective Symptoms, and Objective Findings are required!")
            return
        
        assessment = Assessment(findings, date.today(), subjective_symptoms, objective_findings, history or "N/A")
        self.current_patient.add_assessment(assessment)
        messagebox.showinfo("Success", "Assessment added!")
        
        self.assessment_findings_entry.delete("1.0", tk.END)
        self.subjective_symptoms_entry.delete("1.0", tk.END)
        self.objective_findings_entry.delete("1.0", tk.END)
        self.history_entry.delete("1.0", tk.END)

    def add_reassessment(self):
        if not self.current_patient:
            messagebox.showerror("Error", "No patient selected!")
            return
        
        progress_notes = self.reassessment_progress_entry.get("1.0", tk.END).strip()
        objective_findings = self.reassessment_objective_entry.get("1.0", tk.END).strip()
        recommendations = self.recommendations_entry.get("1.0", tk.END).strip()
        
        if not all([progress_notes, objective_findings, recommendations]):
            messagebox.showerror("Error", "Progress Notes, Objective Findings, and Recommendations are required!")
            return
        
        reassessment = Reassessment(date.today(), progress_notes, objective_findings, recommendations)
        self.current_patient.add_reassessment(reassessment)
        messagebox.showinfo("Success", "Reassessment added!")
        
        self.reassessment_progress_entry.delete("1.0", tk.END)
        self.reassessment_objective_entry.delete("1.0", tk.END)
        self.recommendations_entry.delete("1.0", tk.END)

    def show_details(self):
        if not self.current_patient:
            messagebox.showerror("Error", "No patient selected!")
            return
        
        self.details_text.delete(1.0, tk.END)
        self.details_text.insert(tk.END, str(self.current_patient))

    def save_patients(self):
        try:
            patients_data = {pid: patient.to_dict() for pid, patient in self.patients.items()}
            with open("patients.json", "w") as f:
                json.dump(patients_data, f, indent=4, default=date_to_str)
            messagebox.showinfo("Success", "Patients saved to patients.json!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save patients: {str(e)}")

    def load_patients(self):
        try:
            with open("patients.json", "r") as f:
                patients_data = json.load(f, object_hook=str_to_date)
            self.patients = {pid: Patient.from_dict(data) for pid, data in patients_data.items()}
            self.patient_counter = len(self.patients) + 1
            if self.patients:
                self.current_patient = list(self.patients.values())[-1]
            self.update_patient_menu()
        except FileNotFoundError:
            self.patients = {}
            self.update_patient_menu()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load patients: {str(e)}")

    def generate_word_report(self):
        if not self.current_patient:
            messagebox.showerror("Error", "No patient selected!")
            return

        document = Document()
        document.add_heading(f"Physiotherapy Management Report for {self.current_patient.name}", level=0)

        # Header
        document.add_paragraph(f"Name: {self.current_patient.name}")
        document.add_paragraph(f"Date of Birth: {self.current_patient.dob}")
        document.add_paragraph(f"Medical Aid: {self.current_patient.medical_aid}")
        document.add_paragraph(f"Medical Aid Number: {self.current_patient.medical_aid_number}")
        document.add_paragraph(f"Occupation: {self.current_patient.occupation}")
        document.add_paragraph(f"Physiotherapist: {self.current_patient.physiotherapist_name}")
        document.add_paragraph(f"Referral Diagnosis: {self.current_patient.referral_diagnosis}")

        # Greeting
        document.add_paragraph("Dear Dr,")
        document.add_paragraph(f"Thank you for the referral of the above-mentioned patient.")

        # Patient Overview
        age = (datetime.now().date() - self.current_patient.dob).days // 365
        age_str = f"{age}-year-old" if age >= 1 else f"{(datetime.now().date() - self.current_patient.dob).days // 30}-month-old"
        document.add_paragraph(
            f"The patient is a {age_str} {self.current_patient.gender.lower()} who presented "
            f"with {self.current_patient.referral_diagnosis.lower()}."
        )

        # Initial Assessment
        if self.current_patient.assessments:
            latest_assessment = self.current_patient.assessments[-1]
            document.add_heading("Initial Assessment", level=1)
            document.add_paragraph(
                f"On assessment, the patient reported {latest_assessment.subjective_symptoms.lower()}. "
                f"Objective assessment revealed {latest_assessment.objective_findings.lower()}. "
                f"Relevant history includes {latest_assessment.history.lower()}."
            )
        else:
            document.add_paragraph("No assessment recorded.")

        # Treatment
        document.add_heading("Treatment", level=1)
        document.add_paragraph(
            f"Physiotherapy management included {self.current_patient.treatment_plan.interventions.lower()}. "
            f"A home exercise program was provided, including {self.current_patient.treatment_plan.home_exercise_program.lower()}."
        )

        # Reassessment
        if self.current_patient.reassessments:
            latest_reassessment = self.current_patient.reassessments[-1]
            document.add_heading("Reassessment", level=1)
            document.add_paragraph(
                f"On reassessment, the patient reported {latest_reassessment.progress_notes.lower()}. "
                f"Objective findings included {latest_reassessment.objective_findings.lower()}. "
                f"{latest_reassessment.recommendations}."
            )
        else:
            document.add_paragraph("No reassessment recorded.")

        # Closing
        document.add_paragraph("Kind regards,")
        document.add_paragraph(self.current_patient.physiotherapist_name)
        document.add_paragraph("Physiotherapist")

        try:
            filename = f"{self.current_patient.name.replace(' ', '_')}_report.docx"
            document.save(filename)
            messagebox.showinfo("Success", f"Report saved as '{filename}'")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save report: {str(e)}")

if __name__ == "__main__":
    root = tk.Tk()
    app = PhysioApp(root)
    root.mainloop()

In [3]:
import tkinter as tk
from tkinter import messagebox, ttk
from datetime import date, datetime
from typing import List, Optional
from docx import Document
from docx.shared import Inches
import json

# Helper functions for JSON serialization/deserialization
def date_to_str(obj):
    if isinstance(obj, date):
        return obj.isoformat()
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

def str_to_date(dct):
    for key, value in dct.items():
        if key in ["dob", "assessment_date", "reassessment_date"] and value:
            dct[key] = date.fromisoformat(value)
    return dct

class MedicalHistory:
    def __init__(self, conditions: List[str], surgeries: List[str], allergies: List[str]):
        self.conditions = conditions
        self.surgeries = surgeries
        self.allergies = allergies

    def __str__(self):
        return f"Conditions: {', '.join(self.conditions)}, Surgeries: {', '.join(self.surgeries)}, Allergies: {', '.join(self.allergies)}"

    def to_dict(self):
        return {
            "conditions": self.conditions,
            "surgeries": self.surgeries,
            "allergies": self.allergies
        }

class Diagnosis:
    def __init__(self, diagnosis: str, icd_code: str):
        self.diagnosis = diagnosis
        self.icd_code = icd_code

    def __str__(self):
        return f"Diagnosis: {self.diagnosis} (ICD: {self.icd_code})"

    def to_dict(self):
        return {
            "diagnosis": self.diagnosis,
            "icd_code": self.icd_code
        }

class Assessment:
    def __init__(self, findings: str, assessment_date: date, subjective_symptoms: str, objective_findings: str, history: str):
        self.findings = findings
        self.assessment_date = assessment_date
        self.subjective_symptoms = subjective_symptoms
        self.objective_findings = objective_findings
        self.history = history

    def __str__(self):
        return (f"Assessment on {self.assessment_date}: {self.findings}\n"
                f"Subjective: {self.subjective_symptoms}\n"
                f"Objective: {self.objective_findings}\n"
                f"History: {self.history}")

    def to_dict(self):
        return {
            "findings": self.findings,
            "assessment_date": self.assessment_date,
            "subjective_symptoms": self.subjective_symptoms,
            "objective_findings": self.objective_findings,
            "history": self.history
        }

class Reassessment:
    def __init__(self, date: date, progress_notes: str, objective_findings: str, recommendations: str):
        self.reassessment_date = date
        self.progress_notes = progress_notes
        self.objective_findings = objective_findings
        self.recommendations = recommendations

    def __str__(self):
        return (f"Reassessment on {self.reassessment_date}: {self.progress_notes}\n"
                f"Objective: {self.objective_findings}\n"
                f"Recommendations: {self.recommendations}")

    def to_dict(self):
        return {
            "reassessment_date": self.reassessment_date,
            "progress_notes": self.progress_notes,
            "objective_findings": self.objective_findings,
            "recommendations": self.recommendations
        }

class TreatmentPlan:
    def __init__(self, exercises: List[str], therapies: List[str], modalities: List[str], interventions: str, home_exercise_program: str):
        self.exercises = exercises
        self.therapies = therapies
        self.modalities = modalities
        self.interventions = interventions
        self.home_exercise_program = home_exercise_program

    def __str__(self):
        return (f"Exercises: {', '.join(self.exercises)}\n"
                f"Therapies: {', '.join(self.therapies)}\n"
                f"Modalities: {', '.join(self.modalities)}\n"
                f"Interventions: {self.interventions}\n"
                f"Home Program: {self.home_exercise_program}")

    def to_dict(self):
        return {
            "exercises": self.exercises,
            "therapies": self.therapies,
            "modalities": self.modalities,
            "interventions": self.interventions,
            "home_exercise_program": self.home_exercise_program
        }

class Patient:
    def __init__(self, name: str, contact: str, dob: date, gender: str, medical_aid: str, medical_aid_number: str,
                 occupation: str, referral_diagnosis: str, physiotherapist_name: str, medical_history: MedicalHistory,
                 diagnosis: Diagnosis, treatment_plan: TreatmentPlan):
        self.name = name
        self.contact = contact
        self.dob = dob
        self.gender = gender
        self.medical_aid = medical_aid
        self.medical_aid_number = medical_aid_number
        self.occupation = occupation
        self.referral_diagnosis = referral_diagnosis
        self.physiotherapist_name = physiotherapist_name
        self.medical_history = medical_history
        self.diagnosis = diagnosis
        self.treatment_plan = treatment_plan
        self.assessments: List[Assessment] = []
        self.reassessments: List[Reassessment] = []

    def add_assessment(self, assessment: Assessment):
        self.assessments.append(assessment)

    def add_reassessment(self, reassessment: Reassessment):
        self.reassessments.append(reassessment)

    def __str__(self):
        return (f"Patient: {self.name}, DOB: {self.dob}, Gender: {self.gender}, Contact: {self.contact}\n"
                f"Medical Aid: {self.medical_aid}, Number: {self.medical_aid_number}\n"
                f"Occupation: {self.occupation}, Referral Diagnosis: {self.referral_diagnosis}\n"
                f"Physiotherapist: {self.physiotherapist_name}\n"
                f"Medical History: {self.medical_history}\n"
                f"Diagnosis: {self.diagnosis}\n"
                f"Treatment Plan: {self.treatment_plan}\n"
                f"Assessments: {len(self.assessments)}, Reassessments: {len(self.reassessments)}")

    def to_dict(self):
        return {
            "name": self.name,
            "contact": self.contact,
            "dob": self.dob,
            "gender": self.gender,
            "medical_aid": self.medical_aid,
            "medical_aid_number": self.medical_aid_number,
            "occupation": self.occupation,
            "referral_diagnosis": self.referral_diagnosis,
            "physiotherapist_name": self.physiotherapist_name,
            "medical_history": self.medical_history.to_dict(),
            "diagnosis": self.diagnosis.to_dict(),
            "treatment_plan": self.treatment_plan.to_dict(),
            "assessments": [assessment.to_dict() for assessment in self.assessments],
            "reassessments": [reassessment.to_dict() for reassessment in self.reassessments]
        }

    @classmethod
    def from_dict(cls, data):
        medical_history = MedicalHistory(**data["medical_history"])
        diagnosis = Diagnosis(**data["diagnosis"])
        # Handle legacy TreatmentPlan data
        treatment_plan_data = data["treatment_plan"]
        treatment_plan = TreatmentPlan(
            treatment_plan_data.get("exercises", ["none"]),
            treatment_plan_data.get("therapies", ["none"]),
            treatment_plan_data.get("modalities", ["none"]),
            treatment_plan_data.get("interventions", "Initial treatment"),  # Default for missing field
            treatment_plan_data.get("home_exercise_program", "Home exercises prescribed")  # Default for missing field
        )
        patient = cls(
            data["name"], data["contact"], data["dob"], data["gender"], data["medical_aid"],
            data["medical_aid_number"], data["occupation"], data["referral_diagnosis"],
            data["physiotherapist_name"], medical_history, diagnosis, treatment_plan
        )
        patient.assessments = [Assessment(**assessment) for assessment in data.get("assessments", [])]
        patient.reassessments = [Reassessment(**reassessment) for reassessment in data.get("reassessments", [])]
        return patient

class PhysioApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Physiotherapy Report Generator")
        self.patients = {}
        self.current_patient = None
        self.patient_counter = 1
        self.patient_var = tk.StringVar()
        self.patient_var.set("Select Patient")

        # Create menu bar
        self.menu_bar = tk.Menu(self.root)
        self.root.config(menu=self.menu_bar)

        file_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="File", menu=file_menu)
        file_menu.add_command(label="Save Patients", command=self.save_patients)
        file_menu.add_command(label="Load Patients", command=self.load_patients)
        file_menu.add_separator()
        file_menu.add_command(label="Exit", command=self.root.quit)

        patient_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="Patient Management", menu=patient_menu)
        patient_menu.add_command(label="Add Patient", command=lambda: self.show_frame("patient"))
        patient_menu.add_command(label="View Patient Details", command=self.show_details)

        assess_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="Assessments & Progress", menu=assess_menu)
        assess_menu.add_command(label="Add Assessment", command=lambda: self.show_frame("assessment"))
        assess_menu.add_command(label="Add Reassessment", command=lambda: self.show_frame("assessment"))

        report_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="Reporting", menu=report_menu)
        report_menu.add_command(label="Generate Patient Report", command=self.generate_word_report)

        # Create main container for frames
        self.container = tk.Frame(self.root)
        self.container.grid(row=0, column=0, sticky="nsew")

        self.frames = {}
        self.create_patient_frame()
        self.create_assessment_frame()
        self.show_frame("patient")

        self.load_patients()

    def create_patient_frame(self):
        frame = tk.Frame(self.container)
        self.frames["patient"] = frame

        tk.Label(frame, text="Select Patient:").grid(row=0, column=0, padx=5, pady=5)
        self.patient_menu = tk.OptionMenu(frame, self.patient_var, *self.get_patient_names(), command=self.select_patient)
        self.patient_menu.grid(row=0, column=1, padx=5, pady=5, sticky="ew")

        tk.Label(frame, text="Patient Name:").grid(row=1, column=0, padx=5, pady=5)
        self.name_entry = tk.Entry(frame)
        self.name_entry.grid(row=1, column=1, padx=5, pady=5)

        tk.Label(frame, text="Contact (Email):").grid(row=2, column=0, padx=5, pady=5)
        self.contact_entry = tk.Entry(frame)
        self.contact_entry.grid(row=2, column=1, padx=5, pady=5)

        tk.Label(frame, text="DOB (YYYY-MM-DD):").grid(row=3, column=0, padx=5, pady=5)
        self.dob_entry = tk.Entry(frame)
        self.dob_entry.grid(row=3, column=1, padx=5, pady=5)

        tk.Label(frame, text="Gender:").grid(row=4, column=0, padx=5, pady=5)
        self.gender_entry = tk.Entry(frame)
        self.gender_entry.grid(row=4, column=1, padx=5, pady=5)

        tk.Label(frame, text="Medical Aid:").grid(row=5, column=0, padx=5, pady=5)
        self.medical_aid_entry = tk.Entry(frame)
        self.medical_aid_entry.grid(row=5, column=1, padx=5, pady=5)

        tk.Label(frame, text="Medical Aid Number:").grid(row=6, column=0, padx=5, pady=5)
        self.medical_aid_number_entry = tk.Entry(frame)
        self.medical_aid_number_entry.grid(row=6, column=1, padx=5, pady=5)

        tk.Label(frame, text="Occupation:").grid(row=7, column=0, padx=5, pady=5)
        self.occupation_entry = tk.Entry(frame)
        self.occupation_entry.grid(row=7, column=1, padx=5, pady=5)

        tk.Label(frame, text="Referral Diagnosis:").grid(row=8, column=0, padx=5, pady=5)
        self.referral_diagnosis_entry = tk.Entry(frame)
        self.referral_diagnosis_entry.grid(row=8, column=1, padx=5, pady=5)

        tk.Label(frame, text="Physiotherapist Name:").grid(row=9, column=0, padx=5, pady=5)
        self.physiotherapist_name_entry = tk.Entry(frame)
        self.physiotherapist_name_entry.grid(row=9, column=1, padx=5, pady=5)

        tk.Button(frame, text="Add Patient", command=self.add_patient).grid(row=10, column=0, columnspan=2, pady=10)
        self.details_text = tk.Text(frame, height=10, width=50)
        self.details_text.grid(row=11, column=0, columnspan=2, padx=5, pady=5)

    def create_assessment_frame(self):
        frame = tk.Frame(self.container)
        self.frames["assessment"] = frame

        tk.Label(frame, text="Select Patient:").grid(row=0, column=0, padx=5, pady=5)
        self.assess_patient_menu = tk.OptionMenu(frame, self.patient_var, *self.get_patient_names(), command=self.select_patient)
        self.assess_patient_menu.grid(row=0, column=1, padx=5, pady=5, sticky="ew")

        tk.Label(frame, text="Assessment Findings:").grid(row=1, column=0, padx=5, pady=5)
        self.assessment_findings_entry = tk.Text(frame, height=3, width=40)
        self.assessment_findings_entry.grid(row=1, column=1, padx=5, pady=5)

        tk.Label(frame, text="Subjective Symptoms:").grid(row=2, column=0, padx=5, pady=5)
        self.subjective_symptoms_entry = tk.Text(frame, height=3, width=40)
        self.subjective_symptoms_entry.grid(row=2, column=1, padx=5, pady=5)

        tk.Label(frame, text="Objective Findings:").grid(row=3, column=0, padx=5, pady=5)
        self.objective_findings_entry = tk.Text(frame, height=3, width=40)
        self.objective_findings_entry.grid(row=3, column=1, padx=5, pady=5)

        tk.Label(frame, text="History:").grid(row=4, column=0, padx=5, pady=5)
        self.history_entry = tk.Text(frame, height=3, width=40)
        self.history_entry.grid(row=4, column=1, padx=5, pady=5)

        tk.Button(frame, text="Add Assessment", command=self.add_assessment).grid(row=5, column=0, columnspan=2, pady=10)

        tk.Label(frame, text="Reassessment Progress Notes:").grid(row=6, column=0, padx=5, pady=5)
        self.reassessment_progress_entry = tk.Text(frame, height=3, width=40)
        self.reassessment_progress_entry.grid(row=6, column=1, padx=5, pady=5)

        tk.Label(frame, text="Reassessment Objective Findings:").grid(row=7, column=0, padx=5, pady=5)
        self.reassessment_objective_entry = tk.Text(frame, height=3, width=40)
        self.reassessment_objective_entry.grid(row=7, column=1, padx=5, pady=5)

        tk.Label(frame, text="Recommendations:").grid(row=8, column=0, padx=5, pady=5)
        self.recommendations_entry = tk.Text(frame, height=3, width=40)
        self.recommendations_entry.grid(row=8, column=1, padx=5, pady=5)

        tk.Button(frame, text="Add Reassessment", command=self.add_reassessment).grid(row=9, column=0, columnspan=2, pady=10)

    def show_frame(self, frame_name):
        frame = self.frames.get(frame_name)
        if frame:
            frame.grid(row=0, column=0, sticky="nsew")
            for other_frame in self.frames.values():
                if other_frame != frame:
                    other_frame.grid_remove()

    def get_patient_names(self):
        return [f"{pid}: {patient.name}" for pid, patient in self.patients.items()] or ["No patients available"]

    def select_patient(self, *args):
        selected = self.patient_var.get()
        if selected == "No patients available" or selected == "Select Patient":
            self.current_patient = None
            self.details_text.delete(1.0, tk.END)
            self.name_entry.delete(0, tk.END)
            self.contact_entry.delete(0, tk.END)
            self.dob_entry.delete(0, tk.END)
            self.gender_entry.delete(0, tk.END)
            self.medical_aid_entry.delete(0, tk.END)
            self.medical_aid_number_entry.delete(0, tk.END)
            self.occupation_entry.delete(0, tk.END)
            self.referral_diagnosis_entry.delete(0, tk.END)
            self.physiotherapist_name_entry.delete(0, tk.END)
            return
        
        pid = selected.split(":")[0].strip()
        if pid not in self.patients:
            messagebox.showerror("Error", "Invalid patient selection!")
            self.current_patient = None
            return
        
        self.current_patient = self.patients[pid]
        self.name_entry.delete(0, tk.END)
        self.name_entry.insert(0, self.current_patient.name)
        self.contact_entry.delete(0, tk.END)
        self.contact_entry.insert(0, self.current_patient.contact)
        self.dob_entry.delete(0, tk.END)
        self.dob_entry.insert(0, self.current_patient.dob.isoformat())
        self.gender_entry.delete(0, tk.END)
        self.gender_entry.insert(0, self.current_patient.gender)
        self.medical_aid_entry.delete(0, tk.END)
        self.medical_aid_entry.insert(0, self.current_patient.medical_aid)
        self.medical_aid_number_entry.delete(0, tk.END)
        self.medical_aid_number_entry.insert(0, self.current_patient.medical_aid_number)
        self.occupation_entry.delete(0, tk.END)
        self.occupation_entry.insert(0, self.current_patient.occupation)
        self.referral_diagnosis_entry.delete(0, tk.END)
        self.referral_diagnosis_entry.insert(0, self.current_patient.referral_diagnosis)
        self.physiotherapist_name_entry.delete(0, tk.END)
        self.physiotherapist_name_entry.insert(0, self.current_patient.physiotherapist_name)
        self.show_details()

    def update_patient_menu(self):
        for menu in [self.patient_menu, self.assess_patient_menu]:
            menu["menu"].delete(0, "end")
            for name in self.get_patient_names():
                menu["menu"].add_command(label=name, command=lambda n=name: self.patient_var.set(n))
        if self.current_patient:
            for pid, patient in self.patients.items():
                if patient == self.current_patient:
                    self.patient_var.set(f"{pid}: {patient.name}")
                    break
        else:
            self.patient_var.set("Select Patient")

    def add_patient(self):
        try:
            name = self.name_entry.get()
            contact = self.contact_entry.get()
            dob_str = self.dob_entry.get()
            gender = self.gender_entry.get()
            medical_aid = self.medical_aid_entry.get()
            medical_aid_number = self.medical_aid_number_entry.get()
            occupation = self.occupation_entry.get()
            referral_diagnosis = self.referral_diagnosis_entry.get()
            physiotherapist_name = self.physiotherapist_name_entry.get()
            
            if not all([name, contact, dob_str, gender, referral_diagnosis, physiotherapist_name]):
                messagebox.showerror("Error", "Required fields: Name, Contact, DOB, Gender, Referral Diagnosis, Physiotherapist Name!")
                return
            
            try:
                dob = date.fromisoformat(dob_str)
            except ValueError:
                messagebox.showerror("Error", "Invalid DOB format! Use YYYY-MM-DD")
                return

            medical_history = MedicalHistory(["none"], ["none"], ["none"])
            diagnosis = Diagnosis(referral_diagnosis, "N/A")
            treatment_plan = TreatmentPlan(["none"], ["none"], ["none"], "Initial treatment", "Home exercises prescribed")

            self.current_patient = Patient(
                name, contact, dob, gender, medical_aid or "N/A", medical_aid_number or "N/A",
                occupation or "N/A", referral_diagnosis, physiotherapist_name, medical_history,
                diagnosis, treatment_plan
            )
            patient_id = f"patient_{self.patient_counter:03d}"
            self.patients[patient_id] = self.current_patient
            self.patient_counter += 1

            self.update_patient_menu()
            messagebox.showinfo("Success", f"Patient {name} added with ID {patient_id}!")
            
            self.name_entry.delete(0, tk.END)
            self.contact_entry.delete(0, tk.END)
            self.dob_entry.delete(0, tk.END)
            self.gender_entry.delete(0, tk.END)
            self.medical_aid_entry.delete(0, tk.END)
            self.medical_aid_number_entry.delete(0, tk.END)
            self.occupation_entry.delete(0, tk.END)
            self.referral_diagnosis_entry.delete(0, tk.END)
            self.physiotherapist_name_entry.delete(0, tk.END)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to add patient: {str(e)}")

    def add_assessment(self):
        if not self.current_patient:
            messagebox.showerror("Error", "No patient selected!")
            return
        
        findings = self.assessment_findings_entry.get("1.0", tk.END).strip()
        subjective_symptoms = self.subjective_symptoms_entry.get("1.0", tk.END).strip()
        objective_findings = self.objective_findings_entry.get("1.0", tk.END).strip()
        history = self.history_entry.get("1.0", tk.END).strip()
        
        if not all([findings, subjective_symptoms, objective_findings]):
            messagebox.showerror("Error", "Findings, Subjective Symptoms, and Objective Findings are required!")
            return
        
        assessment = Assessment(findings, date.today(), subjective_symptoms, objective_findings, history or "N/A")
        self.current_patient.add_assessment(assessment)
        messagebox.showinfo("Success", "Assessment added!")
        
        self.assessment_findings_entry.delete("1.0", tk.END)
        self.subjective_symptoms_entry.delete("1.0", tk.END)
        self.objective_findings_entry.delete("1.0", tk.END)
        self.history_entry.delete("1.0", tk.END)

    def add_reassessment(self):
        if not self.current_patient:
            messagebox.showerror("Error", "No patient selected!")
            return
        
        progress_notes = self.reassessment_progress_entry.get("1.0", tk.END).strip()
        objective_findings = self.reassessment_objective_entry.get("1.0", tk.END).strip()
        recommendations = self.recommendations_entry.get("1.0", tk.END).strip()
        
        if not all([progress_notes, objective_findings, recommendations]):
            messagebox.showerror("Error", "Progress Notes, Objective Findings, and Recommendations are required!")
            return
        
        reassessment = Reassessment(date.today(), progress_notes, objective_findings, recommendations)
        self.current_patient.add_reassessment(reassessment)
        messagebox.showinfo("Success", "Reassessment added!")
        
        self.reassessment_progress_entry.delete("1.0", tk.END)
        self.reassessment_objective_entry.delete("1.0", tk.END)
        self.recommendations_entry.delete("1.0", tk.END)

    def show_details(self):
        if not self.current_patient:
            messagebox.showerror("Error", "No patient selected!")
            return
        
        self.details_text.delete(1.0, tk.END)
        self.details_text.insert(tk.END, str(self.current_patient))

    def save_patients(self):
        try:
            patients_data = {pid: patient.to_dict() for pid, patient in self.patients.items()}
            with open("patients.json", "w") as f:
                json.dump(patients_data, f, indent=4, default=date_to_str)
            messagebox.showinfo("Success", "Patients saved to patients.json!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save patients: {str(e)}")

    def load_patients(self):
        try:
            with open("patients.json", "r") as f:
                patients_data = json.load(f, object_hook=str_to_date)
            self.patients = {pid: Patient.from_dict(data) for pid, data in patients_data.items()}
            self.patient_counter = len(self.patients) + 1
            if self.patients:
                self.current_patient = list(self.patients.values())[-1]
            self.update_patient_menu()
        except FileNotFoundError:
            self.patients = {}
            self.update_patient_menu()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load patients: {str(e)}")

    def generate_word_report(self):
        if not self.current_patient:
            messagebox.showerror("Error", "No patient selected!")
            return

        document = Document()
        document.add_heading(f"Physiotherapy Management Report for {self.current_patient.name}", level=0)

        # Header
        document.add_paragraph(f"Name: {self.current_patient.name}")
        document.add_paragraph(f"Date of Birth: {self.current_patient.dob}")
        document.add_paragraph(f"Medical Aid: {self.current_patient.medical_aid}")
        document.add_paragraph(f"Medical Aid Number: {self.current_patient.medical_aid_number}")
        document.add_paragraph(f"Occupation: {self.current_patient.occupation}")
        document.add_paragraph(f"Physiotherapist: {self.current_patient.physiotherapist_name}")
        document.add_paragraph(f"Referral Diagnosis: {self.current_patient.referral_diagnosis}")

        # Greeting
        document.add_paragraph("Dear Dr,")
        document.add_paragraph(f"Thank you for the referral of the above-mentioned patient.")

        # Patient Overview
        age = (datetime.now().date() - self.current_patient.dob).days // 365
        age_str = f"{age}-year-old" if age >= 1 else f"{(datetime.now().date() - self.current_patient.dob).days // 30}-month-old"
        document.add_paragraph(
            f"The patient is a {age_str} {self.current_patient.gender.lower()} who presented "
            f"with {self.current_patient.referral_diagnosis.lower()}."
        )

        # Initial Assessment
        if self.current_patient.assessments:
            latest_assessment = self.current_patient.assessments[-1]
            document.add_heading("Initial Assessment", level=1)
            document.add_paragraph(
                f"On assessment, the patient reported {latest_assessment.subjective_symptoms.lower()}. "
                f"Objective assessment revealed {latest_assessment.objective_findings.lower()}. "
                f"Relevant history includes {latest_assessment.history.lower()}."
            )
        else:
            document.add_paragraph("No assessment recorded.")

        # Treatment
        document.add_heading("Treatment", level=1)
        document.add_paragraph(
            f"Physiotherapy management included {self.current_patient.treatment_plan.interventions.lower()}. "
            f"A home exercise program was provided, including {self.current_patient.treatment_plan.home_exercise_program.lower()}."
        )

        # Reassessment
        if self.current_patient.reassessments:
            latest_reassessment = self.current_patient.reassessments[-1]
            document.add_heading("Reassessment", level=1)
            document.add_paragraph(
                f"On reassessment, the patient reported {latest_reassessment.progress_notes.lower()}. "
                f"Objective findings included {latest_reassessment.objective_findings.lower()}. "
                f"{latest_reassessment.recommendations}."
            )
        else:
            document.add_paragraph("No reassessment recorded.")

        # Closing
        document.add_paragraph("Kind regards,")
        document.add_paragraph(self.current_patient.physiotherapist_name)
        document.add_paragraph("Physiotherapist")

        try:
            filename = f"{self.current_patient.name.replace(' ', '_')}_report.docx"
            document.save(filename)
            messagebox.showinfo("Success", f"Report saved as '{filename}'")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save report: {str(e)}")

if __name__ == "__main__":
    root = tk.Tk()
    app = PhysioApp(root)
    root.mainloop()

In [5]:
import tkinter as tk
from tkinter import messagebox, ttk
from datetime import date, datetime
from typing import List, Optional
from docx import Document
from docx.shared import Inches
import json

# Helper functions for JSON serialization/deserialization
def date_to_str(obj):
    if isinstance(obj, date):
        return obj.isoformat()
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

def str_to_date(dct):
    # Recursively process dictionaries to convert date strings to date objects
    if isinstance(dct, dict):
        for key, value in dct.items():
            if key in ["dob", "assessment_date", "reassessment_date"] and isinstance(value, str):
                try:
                    dct[key] = date.fromisoformat(value)
                except ValueError:
                    print(f"Warning: Invalid date format for {key}: {value}")
            elif isinstance(value, dict):
                str_to_date(value)
            elif isinstance(value, list):
                for item in value:
                    str_to_date(item)
    return dct

class MedicalHistory:
    def __init__(self, conditions: List[str], surgeries: List[str], allergies: List[str]):
        self.conditions = conditions
        self.surgeries = surgeries
        self.allergies = allergies

    def __str__(self):
        return f"Conditions: {', '.join(self.conditions)}, Surgeries: {', '.join(self.surgeries)}, Allergies: {', '.join(self.allergies)}"

    def to_dict(self):
        return {
            "conditions": self.conditions,
            "surgeries": self.surgeries,
            "allergies": self.allergies
        }

class Diagnosis:
    def __init__(self, diagnosis: str, icd_code: str):
        self.diagnosis = diagnosis
        self.icd_code = icd_code

    def __str__(self):
        return f"Diagnosis: {self.diagnosis} (ICD: {self.icd_code})"

    def to_dict(self):
        return {
            "diagnosis": self.diagnosis,
            "icd_code": self.icd_code
        }

class Assessment:
    def __init__(self, findings: str, assessment_date: date, subjective_symptoms: str, objective_findings: str, history: str):
        self.findings = findings
        self.assessment_date = assessment_date
        self.subjective_symptoms = subjective_symptoms
        self.objective_findings = objective_findings
        self.history = history

    def __str__(self):
        return (f"Assessment on {self.assessment_date}: {self.findings}\n"
                f"Subjective: {self.subjective_symptoms}\n"
                f"Objective: {self.objective_findings}\n"
                f"History: {self.history}")

    def to_dict(self):
        return {
            "findings": self.findings,
            "assessment_date": self.assessment_date,
            "subjective_symptoms": self.subjective_symptoms,
            "objective_findings": self.objective_findings,
            "history": self.history
        }

class Reassessment:
    def __init__(self, date: date, progress_notes: str, objective_findings: str, recommendations: str):
        self.reassessment_date = date
        self.progress_notes = progress_notes
        self.objective_findings = objective_findings
        self.recommendations = recommendations

    def __str__(self):
        return (f"Reassessment on {self.reassessment_date}: {self.progress_notes}\n"
                f"Objective: {self.objective_findings}\n"
                f"Recommendations: {self.recommendations}")

    def to_dict(self):
        return {
            "reassessment_date": self.reassessment_date,
            "progress_notes": self.progress_notes,
            "objective_findings": self.objective_findings,
            "recommendations": self.recommendations
        }

class TreatmentPlan:
    def __init__(self, exercises: List[str], therapies: List[str], modalities: List[str], interventions: str, home_exercise_program: str):
        self.exercises = exercises
        self.therapies = therapies
        self.modalities = modalities
        self.interventions = interventions
        self.home_exercise_program = home_exercise_program

    def __str__(self):
        return (f"Exercises: {', '.join(self.exercises)}\n"
                f"Therapies: {', '.join(self.therapies)}\n"
                f"Modalities: {', '.join(self.modalities)}\n"
                f"Interventions: {self.interventions}\n"
                f"Home Program: {self.home_exercise_program}")

    def to_dict(self):
        return {
            "exercises": self.exercises,
            "therapies": self.therapies,
            "modalities": self.modalities,
            "interventions": self.interventions,
            "home_exercise_program": self.home_exercise_program
        }

class Patient:
    def __init__(self, name: str, contact: str, dob: date, gender: str, medical_aid: str, medical_aid_number: str,
                 occupation: str, referral_diagnosis: str, physiotherapist_name: str, medical_history: MedicalHistory,
                 diagnosis: Diagnosis, treatment_plan: TreatmentPlan):
        self.name = name
        self.contact = contact
        self.dob = dob
        self.gender = gender
        self.medical_aid = medical_aid
        self.medical_aid_number = medical_aid_number
        self.occupation = occupation
        self.referral_diagnosis = referral_diagnosis
        self.physiotherapist_name = physiotherapist_name
        self.medical_history = medical_history
        self.diagnosis = diagnosis
        self.treatment_plan = treatment_plan
        self.assessments: List[Assessment] = []
        self.reassessments: List[Reassessment] = []

    def add_assessment(self, assessment: Assessment):
        self.assessments.append(assessment)

    def add_reassessment(self, reassessment: Reassessment):
        self.reassessments.append(reassessment)

    def __str__(self):
        return (f"Patient: {self.name}, DOB: {self.dob}, Gender: {self.gender}, Contact: {self.contact}\n"
                f"Medical Aid: {self.medical_aid}, Number: {self.medical_aid_number}\n"
                f"Occupation: {self.occupation}, Referral Diagnosis: {self.referral_diagnosis}\n"
                f"Physiotherapist: {self.physiotherapist_name}\n"
                f"Medical History: {self.medical_history}\n"
                f"Diagnosis: {self.diagnosis}\n"
                f"Treatment Plan: {self.treatment_plan}\n"
                f"Assessments: {len(self.assessments)}, Reassessments: {len(self.reassessments)}")

    def to_dict(self):
        return {
            "name": self.name,
            "contact": self.contact,
            "dob": self.dob,
            "gender": self.gender,
            "medical_aid": self.medical_aid,
            "medical_aid_number": self.medical_aid_number,
            "occupation": self.occupation,
            "referral_diagnosis": self.referral_diagnosis,
            "physiotherapist_name": self.physiotherapist_name,
            "medical_history": self.medical_history.to_dict(),
            "diagnosis": self.diagnosis.to_dict(),
            "treatment_plan": self.treatment_plan.to_dict(),
            "assessments": [assessment.to_dict() for assessment in self.assessments],
            "reassessments": [reassessment.to_dict() for reassessment in self.reassessments]
        }

    @classmethod
    def from_dict(cls, data):
        medical_history = MedicalHistory(**data["medical_history"])
        diagnosis = Diagnosis(**data["diagnosis"])
        treatment_plan_data = data["treatment_plan"]
        treatment_plan = TreatmentPlan(
            treatment_plan_data.get("exercises", ["none"]),
            treatment_plan_data.get("therapies", ["none"]),
            treatment_plan_data.get("modalities", ["none"]),
            treatment_plan_data.get("interventions", "Initial treatment"),
            treatment_plan_data.get("home_exercise_program", "Home exercises prescribed")
        )
        patient = cls(
            data["name"], data["contact"], data["dob"], data["gender"], data["medical_aid"],
            data["medical_aid_number"], data["occupation"], data["referral_diagnosis"],
            data["physiotherapist_name"], medical_history, diagnosis, treatment_plan
        )
        patient.assessments = [Assessment(**assessment) for assessment in data.get("assessments", [])]
        patient.reassessments = [Reassessment(**reassessment) for reassessment in data.get("reassessments", [])]
        return patient

class PhysioApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Physiotherapy Report Generator")
        self.patients = {}
        self.current_patient = None
        self.patient_counter = 1
        self.patient_var = tk.StringVar()
        self.patient_var.set("Select Patient")

        # Create menu bar
        self.menu_bar = tk.Menu(self.root)
        self.root.config(menu=self.menu_bar)

        file_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="File", menu=file_menu)
        file_menu.add_command(label="Save Patients", command=self.save_patients)
        file_menu.add_command(label="Load Patients", command=self.load_patients)
        file_menu.add_separator()
        file_menu.add_command(label="Exit", command=self.root.quit)

        patient_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="Patient Management", menu=patient_menu)
        patient_menu.add_command(label="Add Patient", command=lambda: self.show_frame("patient"))
        patient_menu.add_command(label="View Patient Details", command=self.show_details)

        assess_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="Assessments & Progress", menu=assess_menu)
        assess_menu.add_command(label="Add Assessment", command=lambda: self.show_frame("assessment"))
        assess_menu.add_command(label="Add Reassessment", command=lambda: self.show_frame("assessment"))

        report_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="Reporting", menu=report_menu)
        report_menu.add_command(label="Generate Patient Report", command=self.generate_word_report)

        # Create main container for frames
        self.container = tk.Frame(self.root)
        self.container.grid(row=0, column=0, sticky="nsew")

        self.frames = {}
        self.create_patient_frame()
        self.create_assessment_frame()
        self.show_frame("patient")

        self.load_patients()

    def create_patient_frame(self):
        frame = tk.Frame(self.container)
        self.frames["patient"] = frame

        tk.Label(frame, text="Select Patient:").grid(row=0, column=0, padx=5, pady=5)
        self.patient_menu = tk.OptionMenu(frame, self.patient_var, *self.get_patient_names(), command=self.select_patient)
        self.patient_menu.grid(row=0, column=1, padx=5, pady=5, sticky="ew")

        tk.Label(frame, text="Patient Name:").grid(row=1, column=0, padx=5, pady=5)
        self.name_entry = tk.Entry(frame)
        self.name_entry.grid(row=1, column=1, padx=5, pady=5)

        tk.Label(frame, text="Contact (Email):").grid(row=2, column=0, padx=5, pady=5)
        self.contact_entry = tk.Entry(frame)
        self.contact_entry.grid(row=2, column=1, padx=5, pady=5)

        tk.Label(frame, text="DOB (YYYY-MM-DD):").grid(row=3, column=0, padx=5, pady=5)
        self.dob_entry = tk.Entry(frame)
        self.dob_entry.grid(row=3, column=1, padx=5, pady=5)

        tk.Label(frame, text="Gender:").grid(row=4, column=0, padx=5, pady=5)
        self.gender_entry = tk.Entry(frame)
        self.gender_entry.grid(row=4, column=1, padx=5, pady=5)

        tk.Label(frame, text="Medical Aid:").grid(row=5, column=0, padx=5, pady=5)
        self.medical_aid_entry = tk.Entry(frame)
        self.medical_aid_entry.grid(row=5, column=1, padx=5, pady=5)

        tk.Label(frame, text="Medical Aid Number:").grid(row=6, column=0, padx=5, pady=5)
        self.medical_aid_number_entry = tk.Entry(frame)
        self.medical_aid_number_entry.grid(row=6, column=1, padx=5, pady=5)

        tk.Label(frame, text="Occupation:").grid(row=7, column=0, padx=5, pady=5)
        self.occupation_entry = tk.Entry(frame)
        self.occupation_entry.grid(row=7, column=1, padx=5, pady=5)

        tk.Label(frame, text="Referral Diagnosis:").grid(row=8, column=0, padx=5, pady=5)
        self.referral_diagnosis_entry = tk.Entry(frame)
        self.referral_diagnosis_entry.grid(row=8, column=1, padx=5, pady=5)

        tk.Label(frame, text="Physiotherapist Name:").grid(row=9, column=0, padx=5, pady=5)
        self.physiotherapist_name_entry = tk.Entry(frame)
        self.physiotherapist_name_entry.grid(row=9, column=1, padx=5, pady=5)

        tk.Button(frame, text="Add Patient", command=self.add_patient).grid(row=10, column=0, columnspan=2, pady=10)
        self.details_text = tk.Text(frame, height=10, width=50)
        self.details_text.grid(row=11, column=0, columnspan=2, padx=5, pady=5)

    def create_assessment_frame(self):
        frame = tk.Frame(self.container)
        self.frames["assessment"] = frame

        tk.Label(frame, text="Select Patient:").grid(row=0, column=0, padx=5, pady=5)
        self.assess_patient_menu = tk.OptionMenu(frame, self.patient_var, *self.get_patient_names(), command=self.select_patient)
        self.assess_patient_menu.grid(row=0, column=1, padx=5, pady=5, sticky="ew")

        tk.Label(frame, text="Assessment Findings:").grid(row=1, column=0, padx=5, pady=5)
        self.assessment_findings_entry = tk.Text(frame, height=3, width=40)
        self.assessment_findings_entry.grid(row=1, column=1, padx=5, pady=5)

        tk.Label(frame, text="Subjective Symptoms:").grid(row=2, column=0, padx=5, pady=5)
        self.subjective_symptoms_entry = tk.Text(frame, height=3, width=40)
        self.subjective_symptoms_entry.grid(row=2, column=1, padx=5, pady=5)

        tk.Label(frame, text="Objective Findings:").grid(row=3, column=0, padx=5, pady=5)
        self.objective_findings_entry = tk.Text(frame, height=3, width=40)
        self.objective_findings_entry.grid(row=3, column=1, padx=5, pady=5)

        tk.Label(frame, text="History:").grid(row=4, column=0, padx=5, pady=5)
        self.history_entry = tk.Text(frame, height=3, width=40)
        self.history_entry.grid(row=4, column=1, padx=5, pady=5)

        tk.Button(frame, text="Add Assessment", command=self.add_assessment).grid(row=5, column=0, columnspan=2, pady=10)

        tk.Label(frame, text="Reassessment Progress Notes:").grid(row=6, column=0, padx=5, pady=5)
        self.reassessment_progress_entry = tk.Text(frame, height=3, width=40)
        self.reassessment_progress_entry.grid(row=6, column=1, padx=5, pady=5)

        tk.Label(frame, text="Reassessment Objective Findings:").grid(row=7, column=0, padx=5, pady=5)
        self.reassessment_objective_entry = tk.Text(frame, height=3, width=40)
        self.reassessment_objective_entry.grid(row=7, column=1, padx=5, pady=5)

        tk.Label(frame, text="Recommendations:").grid(row=8, column=0, padx=5, pady=5)
        self.recommendations_entry = tk.Text(frame, height=3, width=40)
        self.recommendations_entry.grid(row=8, column=1, padx=5, pady=5)

        tk.Button(frame, text="Add Reassessment", command=self.add_reassessment).grid(row=9, column=0, columnspan=2, pady=10)

    def show_frame(self, frame_name):
        frame = self.frames.get(frame_name)
        if frame:
            frame.grid(row=0, column=0, sticky="nsew")
            for other_frame in self.frames.values():
                if other_frame != frame:
                    other_frame.grid_remove()

    def get_patient_names(self):
        return [f"{pid}: {patient.name}" for pid, patient in self.patients.items()] or ["No patients available"]

    def select_patient(self, *args):
        selected = self.patient_var.get()
        if selected == "No patients available" or selected == "Select Patient":
            self.current_patient = None
            self.details_text.delete(1.0, tk.END)
            self.name_entry.delete(0, tk.END)
            self.contact_entry.delete(0, tk.END)
            self.dob_entry.delete(0, tk.END)
            self.gender_entry.delete(0, tk.END)
            self.medical_aid_entry.delete(0, tk.END)
            self.medical_aid_number_entry.delete(0, tk.END)
            self.occupation_entry.delete(0, tk.END)
            self.referral_diagnosis_entry.delete(0, tk.END)
            self.physiotherapist_name_entry.delete(0, tk.END)
            return
        
        pid = selected.split(":")[0].strip()
        if pid not in self.patients:
            messagebox.showerror("Error", "Invalid patient selection!")
            self.current_patient = None
            return
        
        self.current_patient = self.patients[pid]
        self.name_entry.delete(0, tk.END)
        self.name_entry.insert(0, self.current_patient.name)
        self.contact_entry.delete(0, tk.END)
        self.contact_entry.insert(0, self.current_patient.contact)
        self.dob_entry.delete(0, tk.END)
        self.dob_entry.insert(0, self.current_patient.dob.isoformat())
        self.gender_entry.delete(0, tk.END)
        self.gender_entry.insert(0, self.current_patient.gender)
        self.medical_aid_entry.delete(0, tk.END)
        self.medical_aid_entry.insert(0, self.current_patient.medical_aid)
        self.medical_aid_number_entry.delete(0, tk.END)
        self.medical_aid_number_entry.insert(0, self.current_patient.medical_aid_number)
        self.occupation_entry.delete(0, tk.END)
        self.occupation_entry.insert(0, self.current_patient.occupation)
        self.referral_diagnosis_entry.delete(0, tk.END)
        self.referral_diagnosis_entry.insert(0, self.current_patient.referral_diagnosis)
        self.physiotherapist_name_entry.delete(0, tk.END)
        self.physiotherapist_name_entry.insert(0, self.current_patient.physiotherapist_name)
        self.show_details()

    def update_patient_menu(self):
        for menu in [self.patient_menu, self.assess_patient_menu]:
            menu["menu"].delete(0, "end")
            for name in self.get_patient_names():
                menu["menu"].add_command(label=name, command=lambda n=name: self.patient_var.set(n))
        if self.current_patient:
            for pid, patient in self.patients.items():
                if patient == self.current_patient:
                    self.patient_var.set(f"{pid}: {patient.name}")
                    break
        else:
            self.patient_var.set("Select Patient")

    def add_patient(self):
        try:
            name = self.name_entry.get()
            contact = self.contact_entry.get()
            dob_str = self.dob_entry.get()
            gender = self.gender_entry.get()
            medical_aid = self.medical_aid_entry.get()
            medical_aid_number = self.medical_aid_number_entry.get()
            occupation = self.occupation_entry.get()
            referral_diagnosis = self.referral_diagnosis_entry.get()
            physiotherapist_name = self.physiotherapist_name_entry.get()
            
            if not all([name, contact, dob_str, gender, referral_diagnosis, physiotherapist_name]):
                messagebox.showerror("Error", "Required fields: Name, Contact, DOB, Gender, Referral Diagnosis, Physiotherapist Name!")
                return
            
            try:
                dob = date.fromisoformat(dob_str)
            except ValueError:
                messagebox.showerror("Error", "Invalid DOB format! Use YYYY-MM-DD")
                return

            medical_history = MedicalHistory(["none"], ["none"], ["none"])
            diagnosis = Diagnosis(referral_diagnosis, "N/A")
            treatment_plan = TreatmentPlan(["none"], ["none"], ["none"], "Initial treatment", "Home exercises prescribed")

            self.current_patient = Patient(
                name, contact, dob, gender, medical_aid or "N/A", medical_aid_number or "N/A",
                occupation or "N/A", referral_diagnosis, physiotherapist_name, medical_history,
                diagnosis, treatment_plan
            )
            patient_id = f"patient_{self.patient_counter:03d}"
            self.patients[patient_id] = self.current_patient
            self.patient_counter += 1

            self.update_patient_menu()
            messagebox.showinfo("Success", f"Patient {name} added with ID {patient_id}!")
            
            self.name_entry.delete(0, tk.END)
            self.contact_entry.delete(0, tk.END)
            self.dob_entry.delete(0, tk.END)
            self.gender_entry.delete(0, tk.END)
            self.medical_aid_entry.delete(0, tk.END)
            self.medical_aid_number_entry.delete(0, tk.END)
            self.occupation_entry.delete(0, tk.END)
            self.referral_diagnosis_entry.delete(0, tk.END)
            self.physiotherapist_name_entry.delete(0, tk.END)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to add patient: {str(e)}")

    def add_assessment(self):
        if not self.current_patient:
            messagebox.showerror("Error", "No patient selected!")
            return
        
        findings = self.assessment_findings_entry.get("1.0", tk.END).strip()
        subjective_symptoms = self.subjective_symptoms_entry.get("1.0", tk.END).strip()
        objective_findings = self.objective_findings_entry.get("1.0", tk.END).strip()
        history = self.history_entry.get("1.0", tk.END).strip()
        
        if not all([findings, subjective_symptoms, objective_findings]):
            messagebox.showerror("Error", "Findings, Subjective Symptoms, and Objective Findings are required!")
            return
        
        assessment = Assessment(findings, date.today(), subjective_symptoms, objective_findings, history or "N/A")
        self.current_patient.add_assessment(assessment)
        messagebox.showinfo("Success", "Assessment added!")
        
        self.assessment_findings_entry.delete("1.0", tk.END)
        self.subjective_symptoms_entry.delete("1.0", tk.END)
        self.objective_findings_entry.delete("1.0", tk.END)
        self.history_entry.delete("1.0", tk.END)

    def add_reassessment(self):
        if not self.current_patient:
            messagebox.showerror("Error", "No patient selected!")
            return
        
        progress_notes = self.reassessment_progress_entry.get("1.0", tk.END).strip()
        objective_findings = self.reassessment_objective_entry.get("1.0", tk.END).strip()
        recommendations = self.recommendations_entry.get("1.0", tk.END).strip()
        
        if not all([progress_notes, objective_findings, recommendations]):
            messagebox.showerror("Error", "Progress Notes, Objective Findings, and Recommendations are required!")
            return
        
        reassessment = Reassessment(date.today(), progress_notes, objective_findings, recommendations)
        self.current_patient.add_reassessment(reassessment)
        messagebox.showinfo("Success", "Reassessment added!")
        
        self.reassessment_progress_entry.delete("1.0", tk.END)
        self.reassessment_objective_entry.delete("1.0", tk.END)
        self.recommendations_entry.delete("1.0", tk.END)

    def show_details(self):
        if not self.current_patient:
            messagebox.showerror("Error", "No patient selected!")
            return
        
        self.details_text.delete(1.0, tk.END)
        self.details_text.insert(tk.END, str(self.current_patient))

    def save_patients(self):
        try:
            patients_data = {pid: patient.to_dict() for pid, patient in self.patients.items()}
            with open("patients.json", "w") as f:
                json.dump(patients_data, f, indent=4, default=date_to_str)
            messagebox.showinfo("Success", "Patients saved to patients.json!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save patients: {str(e)}")

    def load_patients(self):
        try:
            with open("patients.json", "r") as f:
                patients_data = json.load(f, object_hook=str_to_date)
            self.patients = {}
            for pid, data in patients_data.items():
                try:
                    self.patients[pid] = Patient.from_dict(data)
                except Exception as e:
                    print(f"Error loading patient {pid}: {str(e)}")
                    messagebox.showwarning("Warning", f"Failed to load patient {pid}: {str(e)}")
            self.patient_counter = len(self.patients) + 1
            if self.patients:
                self.current_patient = list(self.patients.values())[-1]
            self.update_patient_menu()
            print(f"Loaded {len(self.patients)} patients successfully")
        except FileNotFoundError:
            self.patients = {}
            self.update_patient_menu()
            print("No patients.json found, starting with empty patient list")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load patients: {str(e)}")
            print(f"Load patients error: {str(e)}")

    def generate_word_report(self):
        if not self.current_patient:
            messagebox.showerror("Error", "No patient selected!")
            return

        document = Document()
        document.add_heading(f"Physiotherapy Management Report for {self.current_patient.name}", level=0)

        # Header
        document.add_paragraph(f"Name: {self.current_patient.name}")
        document.add_paragraph(f"Date of Birth: {self.current_patient.dob}")
        document.add_paragraph(f"Medical Aid: {self.current_patient.medical_aid}")
        document.add_paragraph(f"Medical Aid Number: {self.current_patient.medical_aid_number}")
        document.add_paragraph(f"Occupation: {self.current_patient.occupation}")
        document.add_paragraph(f"Physiotherapist: {self.current_patient.physiotherapist_name}")
        document.add_paragraph(f"Referral Diagnosis: {self.current_patient.referral_diagnosis}")

        # Greeting
        document.add_paragraph("Dear Dr,")
        document.add_paragraph(f"Thank you for the referral of the above-mentioned patient.")

        # Patient Overview
        age = (datetime.now().date() - self.current_patient.dob).days // 365
        age_str = f"{age}-year-old" if age >= 1 else f"{(datetime.now().date() - self.current_patient.dob).days // 30}-month-old"
        document.add_paragraph(
            f"The patient is a {age_str} {self.current_patient.gender.lower()} who presented "
            f"with {self.current_patient.referral_diagnosis.lower()}."
        )

        # Initial Assessment
        if self.current_patient.assessments:
            latest_assessment = self.current_patient.assessments[-1]
            document.add_heading("Initial Assessment", level=1)
            document.add_paragraph(
                f"On assessment, the patient reported {latest_assessment.subjective_symptoms.lower()}. "
                f"Objective assessment revealed {latest_assessment.objective_findings.lower()}. "
                f"Relevant history includes {latest_assessment.history.lower()}."
            )
        else:
            document.add_paragraph("No assessment recorded.")

        # Treatment
        document.add_heading("Treatment", level=1)
        document.add_paragraph(
            f"Physiotherapy management included {self.current_patient.treatment_plan.interventions.lower()}. "
            f"A home exercise program was provided, including {self.current_patient.treatment_plan.home_exercise_program.lower()}."
        )

        # Reassessment
        if self.current_patient.reassessments:
            latest_reassessment = self.current_patient.reassessments[-1]
            document.add_heading("Reassessment", level=1)
            document.add_paragraph(
                f"On reassessment, the patient reported {latest_reassessment.progress_notes.lower()}. "
                f"Objective findings included {latest_reassessment.objective_findings.lower()}. "
                f"{latest_reassessment.recommendations}."
            )
        else:
            document.add_paragraph("No reassessment recorded.")

        # Closing
        document.add_paragraph("Kind regards,")
        document.add_paragraph(self.current_patient.physiotherapist_name)
        document.add_paragraph("Physiotherapist")

        try:
            filename = f"{self.current_patient.name.replace(' ', '_')}_report.docx"
            document.save(filename)
            messagebox.showinfo("Success", f"Report saved as '{filename}'")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save report: {str(e)}")

if __name__ == "__main__":
    root = tk.Tk()
    app = PhysioApp(root)
    root.mainloop()

Loaded 1 patients successfully
Error loading patient patient_001: Reassessment.__init__() got an unexpected keyword argument 'reassessment_date'
Loaded 0 patients successfully
